In [1]:
import os
import datetime as dt
import pandas as pd
import csv

### Importing Project Libraries

In [2]:
# add the 'src' directory as one where we can import modules
import sys
import pathlib # __file__ isn't recognized in Jupyter, so we need this to get the root path
path = pathlib.Path.cwd()
PROJ_ROOT = path.parent 
src_dir = str(PROJ_ROOT / 'src')
PROJ_ROOT = str(PROJ_ROOT)
sys.path.append(src_dir)
from data import make_dataset

### Confirm that the data is up to date

In [3]:
usernames = ['liamkl', 'vinoct18', 'vinoct24', 'emily2', 'zombeck']
today = dt.date.today()
usernames = make_dataset.refresh_user_data(usernames, PROJ_ROOT, today)
# Re-pulls raw data and refreshes basic interim data files: users_df.pkl 
# and a bunch for each user: filename + username + '.pkl'
#   contacts_df_, day_comm_log_df_, week_comm_log_df_
#   locations_df_, loc_log_df_, day_loc_log_df_, week_loc_log_df_

Interim users_df datafile exists
Data is up to date!


#### Set Interested Date Indices

In [4]:
date_indices = pd.date_range(today - dt.timedelta(35), today + dt.timedelta(0), freq='W-MON')

### Establish interim data path, open some data files

In [5]:
interim_data_path = os.path.join(PROJ_ROOT,
								"data",
								"interim")
users_df = pd.read_pickle(os.path.join(interim_data_path, 'users_df.pkl'))

raw_data_path = os.path.join(PROJ_ROOT,
                             "data",
                             "raw")
raw_users_df = pd.read_pickle(os.path.join(raw_data_path, 'users_df.pkl'))

# raw_users_df #.head(20)
users_df.head(20)


,date_created,refresh_time,userId,unrated_threshold,risky_threshold,supportive_threshold
username,,,,,,
vinoct24,2018-10-24,2018-11-03,5bd0a014dc3f590016b44f5a,0.0,1.0,3.0
liamkl,2018-09-19,2018-11-03,5ba244e1a56d2d001a1a9281,0.0,1.0,3.0
vinoct18,2018-10-18,2018-11-03,5bc8d771efee4500100d13c0,0.0,1.0,3.0
zombeck,2018-10-03,2018-11-03,5bb4aa17febc7c0010d8f234,0.0,1.0,3.0
emily2,2018-10-04,2018-11-03,5bb6117dfb7a9a001057d1fd,0.0,1.0,3.0


In [6]:
users_df

,date_created,refresh_time,userId,unrated_threshold,risky_threshold,supportive_threshold
username,,,,,,
vinoct24,2018-10-24,2018-11-03,5bd0a014dc3f590016b44f5a,0.0,1.0,3.0
liamkl,2018-09-19,2018-11-03,5ba244e1a56d2d001a1a9281,0.0,1.0,3.0
vinoct18,2018-10-18,2018-11-03,5bc8d771efee4500100d13c0,0.0,1.0,3.0
zombeck,2018-10-03,2018-11-03,5bb4aa17febc7c0010d8f234,0.0,1.0,3.0
emily2,2018-10-04,2018-11-03,5bb6117dfb7a9a001057d1fd,0.0,1.0,3.0


In [7]:
notification_dict = make_dataset.make_notification_data(users_df, usernames, PROJ_ROOT)

5ba244e1a56d2d001a1a9281
5bc8d771efee4500100d13c0
                             timestamp                        type  \
_id                                                                  
5bc8ea71c8de070010a1f8ce  1.539894e+12            sent-sms-warning   
5bc8ef3ac8de070010a1f8d9  1.539895e+12       incoming-call-warning   
5bc8f508c8de070010a1f8e3  1.539897e+12         daily-task-question   
5bc8f85eefee4500100d1405  1.539897e+12  notification-task-question   
5bc8f85eefee4500100d1406  1.539897e+12       outgoing-call-warning   

                                            userId  
_id                                                 
5bc8ea71c8de070010a1f8ce  5bc8d771efee4500100d13c0  
5bc8ef3ac8de070010a1f8d9  5bc8d771efee4500100d13c0  
5bc8f508c8de070010a1f8e3  5bc8d771efee4500100d13c0  
5bc8f85eefee4500100d1405  5bc8d771efee4500100d13c0  
5bc8f85eefee4500100d1406  5bc8d771efee4500100d13c0  


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
notification_dict